# Getting Started with Taipy Core on Notebooks

!!! important "Supported Python versions"

    Taipy requires **Python 3.8** or newer.

Welcome to the **Getting Started** guide for Taipy Core. This tour shows you how to use Taipy Core to orchestrate pipelines. Taipy Core implements a modern backend for any data-driven application based on your business case.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_00/imd_end_interface.png width=700>
</div>

# Taipy Core

Taipy Core is one of the components of Taipy to facilitate pipeline orchestration. There are a lot of reasons for using Taipy Core:

- Taipy Core efficiently manages the execution of your functions/pipelines.

- Taipy Core manages data sources and monitors KPIs.

- Taipy Core provides easy management of multiple pipelines and end-user scenarios, which comes in handy in the context of Machine Learning or Mathematical optimization.

To apprehend the Scenario Management aspect of Taipy, you need to understand four essential concepts.

Each step of the **"Getting Started"** will focus on basic concepts of *Taipy*. Note that every step is dependent on 
the code of the previous one. After completing the last step, you will have the skills to develop your own Taipy 
application. 

## Before we begin

Only Taipy has to be installed. **Taipy** package requires Python 3.8 or newer;



In [0]:
# !pip install taipy


## Using Notebooks


# Configuration and execution
## Four fundamental concepts in Taipy Core:
- Data Nodes: are the translation of variables in Taipy. Data Nodes don't contain the data but know how to retrieve it. They can refer to any data: any Python object (string, int, list, dict, model, dataframe, etc), a Pickle file, a CSV file, an SQL database, etc. They know how to read and write data. You can even write your own custom Data Node if needed to access a particular data format.

- Tasks: are the translation of functions in Taipy.

- Pipelines: are a list of tasks executed with intelligent scheduling created automatically by Taipy. They usually represent a sequence of Tasks/functions corresponding to different algorithms like a simple baseline Algorithm or a more sophisticated Machine-Learning pipeline.

- Scenarios: End-Users very often require modifying various parameters to reflect different business situations. Taipy Scenarios will provide the framework to "play"/"execute" pipelines under different conditions/variations (i.e., data/parameters modified by the end user)


## What is a configuration?

Configuration is the structure or model of what is our scenario. It represents our Direct Acyclic Graph but also how we want our data to be stored or how our code is run. Taipy is able to create multiple instances of this structure with different data thus, we need a way to define it through this configuration step.


Let's create our first configuration and then create our entities to submit through Taipy Studio or direct Python Code.



In [1]:
from taipy import Config
import taipy as tp

# Normal function used by Taipy
def double(nb):
    return nb * 2



<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_01/config_01.svg width=700>
</div>

- Two Data Nodes are being configured ('input' and 'output'). The 'input' Data Node has a _default_data_ put at 21. They will be stored as Pickle files by default, and are unique to their scenario.

- The task links the two Data Nodes through the Python function _double_.

- The pipeline will contain this one task, and the scenario will contain this one pipeline.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_01/config_01.gif width=700>
</div>


=== "Python configuration"

Here is the code to configure a simple scenario.



In [2]:
# Configuration of Data Nodes
input_data_node_cfg = Config.configure_data_node("input", default_data=21)
output_data_node_cfg = Config.configure_data_node("output")

# Configuration of tasks
task_cfg = Config.configure_task("double",
                                 double,
                                 input_data_node_cfg,
                                 output_data_node_cfg)

# Configuration of the pipeline and scenario
pipeline_cfg = Config.configure_pipeline("my_pipeline", [task_cfg])
scenario_cfg = Config.configure_scenario("my_scenario", [pipeline_cfg])


In [3]:
# Run of the Core
tp.Core().run()

# Creation of the scenario and execution
scenario = tp.create_scenario(scenario_cfg)
tp.submit(scenario)

print("Value at the end of task", scenario.output.read())


Results:
```
[2022-12-22 16:20:02,740][Taipy][INFO] job JOB_double_699613f8-7ff4-471b-b36c-d59fb6688905 is completed.
Value at the end of task 42
```


# Basic functions

Let's discuss the basic functions that come along with Taipy.

- `<Data Node>.write(<new value>)`: this is how data can be changed through Taipy. _write_ will change the _last_edit_date_ of the data node, which will influence if a task can be skipped or not.

-`tp.get_scenarios()`: this function returns a list of all the scenarios

-`tp.get(<Taipy object ID>)`: this function returns an entity based on the id of the entity

-`tp.delete(<Taipy object ID>)`: this function deletes the entity and nested elements based on the id of the entity

## Utility of having scenarios

Taipy lets the user create multiple instances of the same configuration. Data can differ between instances and can be used to compare different scenarios.

Data can naturally differ depending on the input Data Nodes or the randomness of functions. Moreover, the user can change them with the _write_ function.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_02/config_02.svg width=700>
</div>



In [4]:
scenario = tp.create_scenario(scenario_cfg, name="Scenario")
tp.submit(scenario)
print("First submit", scenario.output.read())


Results:
```
[2022-12-22 16:20:02,874][Taipy][INFO] job JOB_double_a5ecfa4d-1963-4776-8f68-0859d22970b9 is completed.
First submit 42
```

## _write_ function

Using _write_, data of a Data Node can be changed. The syntax is `<Scenario>.<Pipeline>.<Data Node>.write(value)`. If there is just one pipeline, we can just write `<Scenario>.<Data Node>.write(value)`.




In [5]:
print("Before write", scenario.input.read())
scenario.input.write(54)
print("After write",scenario.input.read())



Results:
```
Before write 21
After write 54
```

The submission of the scenario will update the output values.




In [6]:
tp.submit(scenario)
print("Second submit",scenario.output.read())


Results:
```
[2022-12-22 16:20:03,011][Taipy][INFO] job JOB_double_7eee213f-062c-4d67-b0f8-4b54c04e45e7 is completed.
Second submit 108
```

## Other useful functions

- how to access all the scenarios



In [7]:
print([s.input.read() for s in tp.get_scenarios()])



Results:
```
[21, 54]
```

- get an entity from its id



In [8]:
scenario = tp.get(scenario.id)



- delete an entity though its id. Example: how to delete a scenario.



In [9]:
tp.delete(scenario.id)



# Different types of Data Nodes:

- *[Pickle](https://docs.taipy.io/en/latest/manuals/core/config/data-node-config/#pickle)* (default): Taipy can store and read anykind of data that can be serializable.

- *[CSV](https://docs.taipy.io/en/latest/manuals/core/config/data-node-config/#csv)*: Taipy can read and store any dataframe as a CSV.

- *[JSON](https://docs.taipy.io/en/latest/manuals/core/config/data-node-config/#json)*: Taipy can read and store any JSONable data as a JSON file.

- *[SQL](https://docs.taipy.io/en/latest/manuals/core/config/data-node-config/#sql)*: Taipy can read and store a table or data base.

- *[Generic](https://docs.taipy.io/en/latest/manuals/core/config/data-node-config/#generic)*: Taipy provides a generic Data Node that can read and store any data based on the reding and writing function created by the user.

The execution graph used to explain the different concepts is quite simple.

1) Three Data Nodes:
- _historical data_: initial CSV DataFrame
- _month_data_: DataFrame after the filtering on the month (a _Pandas.DataFrame_ as a Pickle file)
- _nb_of_values_: number of values in this month (int as a Pickle file)

2) Two tasks linking these Data Nodes:
- _filter_: filters on the months of the dataframe
- _count_values_: calculates the number of elements in this month

3) One pipeline in a scenario gathering these two tasks.

<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_03/config_03.svg width=700>
</div>



In [10]:
import datetime as dt
import pandas as pd


In [11]:
def filter_current(df):
    current_month = dt.datetime.now().month
    df['Date'] = pd.to_datetime(df['Date']) 
    df = df[df['Date'].dt.month == current_month]
    return df

def count_values(df):
    return len(df)




<div align="center">
 <img src=https://github.com/Avaiga/taipy-getting-started-core/blob/develop/step_03/config_03.gif width=700>
</div>


=== "Python configuration"



In [12]:
# here is a CSV Data Node
historical_data_cfg = Config.configure_csv_data_node(id="historical_data",
                                                     default_path="time_series.csv")
month_values_cfg =  Config.configure_data_node(id="month_data")
nb_of_values_cfg = Config.configure_data_node(id="nb_of_values")


In [13]:
task_filter_current_cfg = Config.configure_task(id="filter_current",
                                                 function=filter_current,
                                                 input=historical_data_cfg,
                                                 output=month_values_cfg)

task_count_values_cfg = Config.configure_task(id="count_values",
                                                 function=count_values,
                                                 input=month_values_cfg,
                                                 output=nb_of_values_cfg)


In [14]:
pipeline_cfg = Config.configure_pipeline(id="my_pipeline",
                                         task_configs=[task_filter_current_cfg,
                                                       task_count_values_cfg])

scenario_cfg = Config.configure_scenario(id="my_scenario",
                                         pipeline_configs=[pipeline_cfg])

#scenario_cfg = Config.configure_scenario_from_tasks(id="my_scenario",
#                                                    task_configs=[task_filter_current_cfg,
#                                                                  task_count_values_cfg])


In [15]:
tp.Core().run()

scenario_1 = tp.create_scenario(scenario_cfg, creation_date=dt.datetime(2022,10,7), name="Scenario 2022/10/7")
scenario_1.submit()

scenario_2 = tp.create_scenario(scenario_cfg, creation_date=dt.datetime(2022,10,7), name="Scenario 2022/10/7")
scenario_2.submit()
